<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/Copy_of_28_Avg_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SQL**

### Import librares

In [ ]:
import pandas as pd

### Set-up for SQL

In [ ]:
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


##### Load dataset

In [ ]:
monarchs = pd.read_csv('monarchs.csv')
presidents = pd.read_csv('presidents.csv')
prime_minister_terms = pd.read_csv('prime_minister_terms.csv')
prime_ministers = pd.read_csv('prime_ministers.csv')
states = pd.read_csv('states.csv')

##### Upload dataset to SQL

In [ ]:
%sql DROP TABLE IF EXISTS monarchs;
%sql --persist monarchs

%sql DROP TABLE IF EXISTS presidents;
%sql --persist presidents

%sql DROP TABLE IF EXISTS prime_minister_terms;
%sql --persist prime_minister_terms

%sql DROP TABLE IF EXISTS prime_ministers;
%sql --persist prime_ministers

%sql DROP TABLE IF EXISTS states;
%sql --persist states

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted states'

##### View all table

In [ ]:
%%sql SELECT * FROM monarchs LIMIT 1;

 * sqlite://
Done.


index,country,continent,monarch
0,Brunei,Asia,Hassanal Bolkiah


In [ ]:
%%sql SELECT * FROM presidents LIMIT 1;

 * sqlite://
Done.


index,country,continent,president
0,Egypt,Africa,Abdel Fattah el-Sisi


In [ ]:
%%sql SELECT * FROM prime_minister_terms LIMIT 1;

 * sqlite://
Done.


index,prime_minister,pm_start
0,Mostafa Madbouly,2018


In [ ]:
%%sql SELECT * FROM prime_ministers LIMIT 1;

 * sqlite://
Done.


index,country,continent,prime_minister
0,Egypt,Africa,Mostafa Madbouly


In [ ]:
%%sql SELECT * FROM states LIMIT 1;

 * sqlite://
Done.


index,country,continent,indep_year
0,Australia,Oceania,1901


### Home work

Ranking: RANK() Write a query to list all countries along with their continent and the rank of their prime minister based on the start year (pm_start), ordered from the earliest to the latest. Use the RANK() window function to rank the prime ministers within each continent.

In [ ]:
%%sql
SELECT pm.country, pm.continent, pmt.pm_start,
RANK() OVER (PARTITION BY pm.continent ORDER BY pmt.pm_start) AS pm_rank
FROM prime_ministers AS pm
INNER JOIN prime_minister_terms AS pmt
ON pm.prime_minister = pmt.prime_minister
ORDER BY pm_start;

 * sqlite://
Done.


country,continent,pm_start,pm_rank
Brunei,Asia,1984,1
India,Asia,2014,2
Portugal,Europe,2015,1
New Zealand,Oceania,2017,1
Egypt,Africa,2018,1
Australia,Oceania,2018,2
United Kingdom,Europe,2019,2
Oman,Asia,2020,3
Norway,Europe,2021,3
Pakistan,Asia,2022,4


Ranking: DENSE_RANK() Write a query to list all countries along with the rank of their president based on the number of prime ministers the country has had. Use the DENSE_RANK() window function to assign ranks, with ties receiving the same rank.

In [ ]:
%%sql
SELECT s.country,
  (SELECT president
  FROM presidents AS p
  WHERE p.country=s.country
  LIMIT 1) AS president,
  COUNT(pm.prime_minister) AS num_pm,
  DENSE_RANK()
    OVER (ORDER BY COUNT(pm.prime_minister) DESC) AS rank
FROM states AS s
LEFT JOIN prime_ministers AS pm
ON s.country = pm.country
GROUP BY s.country
ORDER BY num_pm DESC;

 * sqlite://
Done.


country,president,num_pm,rank
Portugal,Marcelo Rebelo de Sousa,1,1
Pakistan,Arif Alvi,1,1
Oman,None,1,1
Norway,None,1,1
India,Ram Nath Kovind,1,1
Egypt,Abdel Fattah el-Sisi,1,1
Brunei,None,1,1
Australia,None,1,1
Uruguay,Luis Lacalle Pou,0,2
USA,Joe Biden,0,2


Ranking: ROW_NUMBER() Write a query to list all countries along with their prime ministers and assign a unique row number to each prime minister within each country, ordered by the start year (pm_start). Use the ROW_NUMBER() window function for this purpose.

In [ ]:
%%sql
SELECT country, pm.prime_minister, pm_start,
  ROW_NUMBER() OVER (PARTITION BY country ORDER BY pm_start) AS row_num
FROM prime_minister_terms AS pmt
INNER JOIN prime_ministers AS pm
ON pmt.prime_minister = pm.prime_minister;

 * sqlite://
Done.


country,prime_minister,pm_start,row_num
Australia,Scott Morrison,2018,1
Brunei,Hassanal Bolkiah,1984,1
Egypt,Mostafa Madbouly,2018,1
India,Narendra Modi,2014,1
New Zealand,Jacinda Ardern,2017,1
Norway,Jonas Gahr Støre,2021,1
Oman,Haitham bin Tarik,2020,1
Pakistan,Shehbaz Sharif,2022,1
Portugal,António Costa,2015,1
United Kingdom,Boris Johnson,2019,1


Aggregates: SUM() OVER Write a query to list all countries along with their continent and the total number of prime ministers each continent has had. Use the SUM() window function to calculate the total number of prime ministers across each continent.

In [ ]:
%%sql
SELECT pm.continent, pm.country,
  SUM(1) OVER (PARTITION BY pm.continent) AS total_pm
FROM prime_ministers AS pm;

 * sqlite://
Done.


continent,country,total_pm
Africa,Egypt,1
Asia,Pakistan,4
Asia,India,4
Asia,Brunei,4
Asia,Oman,4
Europe,Portugal,3
Europe,United Kingdom,3
Europe,Norway,3
Oceania,Australia,2
Oceania,New Zealand,2


Aggregates: AVG() OVER Write a query to list all countries along with their prime ministers and the average start year of prime ministers within the same continent. Use the AVG() window function to compute the average pm_start year for each continent.

In [ ]:
%%sql
SELECT pm.continent, pm.country, pmt.prime_minister, pmt.pm_start,
  AVG(pmt.pm_start) OVER (PARTITION BY pm.continent) AS avg_pm_start
FROM prime_ministers AS pm
INNER JOIN prime_minister_terms AS pmt
ON pm.prime_minister = pmt.prime_minister;

 * sqlite://
Done.


continent,country,prime_minister,pm_start,avg_pm_start
Africa,Egypt,Mostafa Madbouly,2018,2018.0
Asia,Pakistan,Shehbaz Sharif,2022,2010.0
Asia,India,Narendra Modi,2014,2010.0
Asia,Brunei,Hassanal Bolkiah,1984,2010.0
Asia,Oman,Haitham bin Tarik,2020,2010.0
Europe,Portugal,António Costa,2015,2018.3333333333333
Europe,United Kingdom,Boris Johnson,2019,2018.3333333333333
Europe,Norway,Jonas Gahr Støre,2021,2018.3333333333333
Oceania,Australia,Scott Morrison,2018,2017.5
Oceania,New Zealand,Jacinda Ardern,2017,2017.5


Aggregates: COUNT() OVER Write a query to list all countries along with the total count of presidents in each continent, showing this count next to each country. Use the COUNT() window function to calculate the total number of presidents within each continent.

In [ ]:
%%sql
SELECT s.country, s.continent,
  COUNT(p.president) OVER (PARTITION BY s.continent) AS total_president
FROM states AS s
LEFT JOIN presidents AS p
ON s.country = p.country;

 * sqlite://
Done.


country,continent,total_president
Egypt,Africa,1
Brunei,Asia,2
India,Asia,2
Oman,Asia,2
Pakistan,Asia,2
Norway,Europe,1
Portugal,Europe,1
USA,North America,1
Australia,Oceania,0
Chile,South America,2


#END